In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from transformers import AutoModelForCausalLM
import torch
import torch.nn as nn
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import seaborn as sns
import copy
import json
from tqdm import tqdm as tqdm


#from ppl_utils import eval_ppl


from peft import LoraConfig, get_peft_model, PeftModel

In [3]:
base_path = "/raid/lgh/aids24/EX1/ex1_llama2_7b_omni_w2_real"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_path,
    #torch_dtype = torch.float32, #torch.float16
    device_map = "cuda",
    trust_remote_code=True
)

In [ ]:
model.model.layers[0]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_path,
    torch_dtype = torch.float32, #torch.float16
    device_map = "cuda",
    trust_remote_code=True
)

#model = PeftModel.from_pretrained(model, adapter_path, is_trainable=True)
for para in model.parameters():
    para.requires_grad = False
model.config.use_cache = False
model.eval()

In [4]:
import safetensors

In [5]:
from safetensors import safe_open

tensors = {}
with safe_open(base_path+"/model.safetensors", framework="pt", device=0) as f:
    for k in f.keys():
        tensors[k] = f.get_tensor(k)

In [6]:
for n in tensors.keys():
    print(n)

lm_head.weight
model.embed_tokens.weight
model.layers.0.input_layernorm.weight
model.layers.0.mlp.down_proj.g_idx
model.layers.0.mlp.down_proj.qweight
model.layers.0.mlp.down_proj.qzeros
model.layers.0.mlp.down_proj.scales
model.layers.0.mlp.gate_proj.g_idx
model.layers.0.mlp.gate_proj.qweight
model.layers.0.mlp.gate_proj.qzeros
model.layers.0.mlp.gate_proj.scales
model.layers.0.mlp.up_proj.g_idx
model.layers.0.mlp.up_proj.qweight
model.layers.0.mlp.up_proj.qzeros
model.layers.0.mlp.up_proj.scales
model.layers.0.post_attention_layernorm.weight
model.layers.0.self_attn.k_proj.g_idx
model.layers.0.self_attn.k_proj.qweight
model.layers.0.self_attn.k_proj.qzeros
model.layers.0.self_attn.k_proj.scales
model.layers.0.self_attn.o_proj.g_idx
model.layers.0.self_attn.o_proj.qweight
model.layers.0.self_attn.o_proj.qzeros
model.layers.0.self_attn.o_proj.scales
model.layers.0.self_attn.q_proj.g_idx
model.layers.0.self_attn.q_proj.qweight
model.layers.0.self_attn.q_proj.qzeros
model.layers.0.self_a

In [9]:
tensors["lm_head.weight"]

tensor([[-0.0039,  0.0032, -0.0071,  ...,  0.0053, -0.0082,  0.0070],
        [-0.0315,  0.0466, -0.0023,  ..., -0.0211,  0.0173,  0.0334],
        [-0.0125,  0.0036,  0.0195,  ..., -0.0271,  0.0143, -0.0082],
        ...,
        [-0.0281, -0.0195, -0.0024,  ...,  0.0123, -0.0117, -0.0237],
        [ 0.0229,  0.0255,  0.0315,  ...,  0.0067, -0.0092, -0.0058],
        [ 0.0080, -0.0088,  0.0063,  ..., -0.0293, -0.0200,  0.0337]],
       device='cuda:0', dtype=torch.float16)

In [8]:
tensors["model.embed_tokens.weight"]

tensor([[ 1.2517e-06, -1.7881e-06, -4.3511e-06,  ...,  8.9407e-07,
         -6.5565e-06,  8.9407e-07],
        [ 1.8616e-03, -3.3722e-03,  3.9864e-04,  ..., -8.3008e-03,
          2.5787e-03, -3.9368e-03],
        [ 1.0986e-02,  9.8877e-03, -5.0964e-03,  ...,  2.5177e-03,
          7.7057e-04, -5.0049e-03],
        ...,
        [-1.3977e-02, -2.7313e-03, -1.9897e-02,  ..., -1.0437e-02,
          9.5825e-03, -1.8005e-03],
        [-1.0742e-02,  9.3384e-03,  1.2939e-02,  ..., -3.3203e-02,
         -1.6357e-02,  3.3875e-03],
        [-8.3008e-03, -4.0588e-03, -1.1063e-03,  ...,  3.4790e-03,
         -1.2939e-02,  3.1948e-05]], device='cuda:0', dtype=torch.float16)

In [7]:
tensors["model.layers.0.mlp.down_proj.qweight"]

tensor([[-1432637530,  1423070549,  1721063833,  ...,  -425284886,
         -1430327635,  1872685802],
        [ 1924038393,  1427526790, -1587128151,  ..., -1381307665,
         -1783010379,  -357193046],
        [-1079010710, -1583306411, -1773051580,  ...,  -356584870,
         -2023237782, -1957041510],
        ...,
        [-1431673253, -1344094806, -1427514982,  ...,   354755717,
           342168167, -2035651925],
        [-1646876996,  1767896730,   988523238,  ...,  1499546965,
         -1247455719, -1164535134],
        [-1515587093,  -573216154,  -169428358,  ..., -1726638826,
            73688614,  1788533353]], device='cuda:0', dtype=torch.int32)

In [ ]:
results = eval_ppl(model, False, "llama2", "cuda", base_path)

In [ ]:
output_dir = "ppl_results/EX1/llama2-7b-omni_w4"

In [ ]:
dumped = json.dumps(
    results, indent=2, ensure_ascii=False
)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

    with open(os.path.join(output_dir, "results.json"), "w") as f:
        f.write(dumped)
        f.close()